In [12]:
import pandas as pd
import sys 
sys.path.append("../") 
from maths import baseball_stats


In [18]:
k = ['playerID', 'birthYear', 'birthMonth', 'birthDay']
import_people = pd.read_csv('../datafiles/People.csv', encoding='latin-1', usecols=k)
people = import_people.copy()
people['birthYear'] = people['birthYear'].fillna(0)
people['birthMonth'] = people['birthMonth'].fillna(0)
people['birthDay'] = people['birthDay'].fillna(0)


people

,playerID,birthYear,birthMonth,birthDay
0,aardsda01,1981.0,12.0,27.0
1,aaronha01,1934.0,2.0,5.0
2,aaronto01,1939.0,8.0,5.0
3,aasedo01,1954.0,9.0,8.0
4,abadan01,1972.0,8.0,25.0
...,...,...,...,...
21005,paysojo99,1903.0,2.0,5.0
21006,galbrjo99,1897.0,8.0,10.0
21007,mcshejo99,1944.0,9.0,11.0
21008,weyerle99,1936.0,9.0,3.0
